# GIFT v2.2 - Variational G2 Metric Extraction

## Overview

This notebook implements a **Physics-Informed Neural Network (PINN)** to solve a **constrained variational problem** on G2 geometry.

**Key insight**: This is NOT a simulation of a pre-existing manifold. It is the numerical resolution of a minimization problem whose solution, if it exists, defines the geometry.

### Mathematical Formulation

Find $\phi \in \Lambda^3_+(\mathbb{R}^7)$ minimizing:
$$F[\phi] = ||d\phi||^2_{L^2} + ||d^*\phi||^2_{L^2}$$

Subject to GIFT v2.2 constraints:

| Constraint | Value | Origin |
|------------|-------|--------|
| $b_2$ | 21 | E8 decomposition |
| $b_3$ | 77 = 35 + 42 | Cohomology split |
| $\det(g)$ | 65/32 | Derived from $h^* = 99$ |
| $\kappa_T$ | 1/61 | Global torsion magnitude |
| $\phi$ positive | $\phi \in G_2$ cone | Valid G2 structure |

### Key Paradigm Shift

| Aspect | Old (TCS-based) | New (Variational) |
|--------|-----------------|-------------------|
| Starting point | "Construct TCS manifold" | "Define constraint system" |
| Role of (21,77) | "Verify these emerge" | "Impose as constraints" |
| Role of PINN | "Approximate known geometry" | "Solve optimization problem" |
| Success criterion | "Match TCS structure" | "Satisfy constraints + minimize F" |

## 1. Setup and Dependencies

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass, field
from typing import Dict, List, Tuple, Optional, Callable
from itertools import combinations
from functools import lru_cache
from pathlib import Path
import math
import time

# Device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device: {device}")
print(f"PyTorch: {torch.__version__}")

## 2. GIFT v2.2 Configuration

All values are **topologically determined** - no adjustable parameters.

In [ ]:
@dataclass
class GIFTConfig:
    """GIFT v2.2 structural parameters (topologically fixed)."""
    
    # Dimensions
    dim: int = 7
    
    # Cohomology (TOPOLOGICAL)
    b2_K7: int = 21          # Harmonic 2-forms
    b3_K7: int = 77          # Harmonic 3-forms (35 local + 42 global)
    h_star: int = 99         # Total: b2 + b3 + 1
    
    # GIFT derived constants (PROVEN/TOPOLOGICAL)
    det_g_target: float = 65/32      # = 2.03125
    kappa_T: float = 1/61            # ~ 0.01639
    sin2_theta_W: float = 3/13       # Weinberg angle
    tau: float = 3472/891            # Hierarchy parameter
    
    # Network architecture
    hidden_dim: int = 256
    n_layers: int = 6
    fourier_features: int = 64
    fourier_scale: float = 2.0
    
    # Training
    batch_size: int = 2048
    learning_rate: float = 1e-3
    
    @property
    def n_phi_components(self) -> int:
        return 35  # C(7,3)

config = GIFTConfig()

print("GIFT v2.2 Structural Parameters")
print("="*50)
print(f"Cohomology: b2={config.b2_K7}, b3={config.b3_K7}, h*={config.h_star}")
print(f"det(g) target: {config.det_g_target} = 65/32")
print(f"kappa_T target: {config.kappa_T:.6f} = 1/61")
print(f"sin^2(theta_W): {config.sin2_theta_W:.6f} = 3/13")
print(f"tau: {config.tau:.6f} = 3472/891")

## 3. G2 Geometry Core

The key operation: extract metric $g_{ij}$ from 3-form $\phi$.

In [ ]:
@lru_cache(maxsize=1)
def get_3form_indices():
    """Get (i,j,k) indices for 3-form components with i<j<k."""
    return tuple(combinations(range(7), 3))

def standard_phi_coefficients():
    """Standard G2 3-form phi_0.
    
    phi_0 = e^{123} + e^{145} + e^{167} + e^{246} - e^{257} - e^{347} - e^{356}
    """
    phi = torch.zeros(35)
    terms = [
        ((0, 1, 2), +1),  # e^{123}
        ((0, 3, 4), +1),  # e^{145}
        ((0, 5, 6), +1),  # e^{167}
        ((1, 3, 5), +1),  # e^{246}
        ((1, 4, 6), -1),  # e^{257}
        ((2, 3, 6), -1),  # e^{347}
        ((2, 4, 5), -1),  # e^{356}
    ]
    all_indices = get_3form_indices()
    for (i, j, k), sign in terms:
        pos = all_indices.index((i, j, k))
        phi[pos] = sign
    return phi

class MetricFromPhi(nn.Module):
    """Extract metric g_ij from 3-form phi.
    
    Formula: g_ij = (1/6) sum_{kl} phi_{ikl} phi_{jkl}
    """
    
    def __init__(self):
        super().__init__()
        self._build_contraction_map()
    
    def _build_contraction_map(self):
        indices_3 = get_3form_indices()
        self.contraction_map = [[[] for _ in range(7)] for _ in range(7)]
        
        for comp_a, (a0, a1, a2) in enumerate(indices_3):
            for comp_b, (b0, b1, b2) in enumerate(indices_3):
                set_a, set_b = {a0, a1, a2}, {b0, b1, b2}
                shared = set_a & set_b
                
                if len(shared) == 2:
                    i_only = (set_a - shared).pop()
                    j_only = (set_b - shared).pop()
                    pos_i = [a0, a1, a2].index(i_only)
                    pos_j = [b0, b1, b2].index(j_only)
                    sign = ((-1) ** pos_i) * ((-1) ** pos_j)
                    self.contraction_map[i_only][j_only].append((comp_a, comp_b, sign))
    
    def forward(self, phi: torch.Tensor) -> torch.Tensor:
        squeeze = phi.dim() == 1
        if squeeze:
            phi = phi.unsqueeze(0)
        
        batch = phi.shape[0]
        g = torch.zeros(batch, 7, 7, device=phi.device)
        
        for i in range(7):
            for j in range(7):
                for a, b, s in self.contraction_map[i][j]:
                    g[:, i, j] += s * phi[:, a] * phi[:, b]
        
        g = g / 6.0
        return g.squeeze(0) if squeeze else g

# Test standard G2
phi_0 = standard_phi_coefficients()
metric_fn = MetricFromPhi()
g_0 = metric_fn(phi_0)

print("Standard G2 3-form phi_0:")
print(f"  Non-zero components: {(phi_0.abs() > 0.5).sum().item()}")
print(f"")
print("Induced metric g(phi_0):")
print(f"  det(g) = {torch.det(g_0).item():.6f}")
print(f"  Eigenvalues: {torch.linalg.eigvalsh(g_0).numpy().round(4)}")

## 4. Neural Network Architecture

In [ ]:
class FourierFeatures(nn.Module):
    """Random Fourier features for smooth positional encoding."""
    
    def __init__(self, in_dim=7, n_features=64, scale=2.0):
        super().__init__()
        B = torch.randn(in_dim, n_features) * scale
        self.register_buffer('B', B)
    
    def forward(self, x):
        proj = 2 * math.pi * x @ self.B
        return torch.cat([torch.sin(proj), torch.cos(proj)], dim=-1)


class G2VariationalNet(nn.Module):
    """PINN for G2 variational problem.
    
    Input: x in R^7 (coordinates)
    Output: phi in R^35 (3-form components)
    """
    
    def __init__(self, config):
        super().__init__()
        self.config = config
        
        # Fourier features
        self.fourier = FourierFeatures(
            in_dim=config.dim,
            n_features=config.fourier_features,
            scale=config.fourier_scale
        )
        
        fourier_dim = 2 * config.fourier_features
        
        # MLP backbone
        layers = []
        in_dim = fourier_dim + config.dim
        for i in range(config.n_layers):
            layers.append(nn.Linear(in_dim, config.hidden_dim))
            layers.append(nn.SiLU())
            if i < config.n_layers - 1:
                layers.append(nn.LayerNorm(config.hidden_dim))
            in_dim = config.hidden_dim
        self.backbone = nn.Sequential(*layers)
        
        # Output head
        self.phi_head = nn.Linear(config.hidden_dim, config.n_phi_components)
        
        # Initialize near standard G2
        self._init_weights()
        
        # Geometry
        self.metric_fn = MetricFromPhi()
        self.register_buffer('phi_0', standard_phi_coefficients())
    
    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight, gain=0.1)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
        # Bias toward standard phi
        with torch.no_grad():
            self.phi_head.bias.copy_(standard_phi_coefficients())
            self.phi_head.weight.mul_(0.01)
    
    def forward(self, x, project_positive=True):
        fourier = self.fourier(x)
        combined = torch.cat([x, fourier], dim=-1)
        features = self.backbone(combined)
        phi = self.phi_head(features)
        
        if project_positive:
            phi = self._soft_project(phi)
        return phi
    
    def _soft_project(self, phi, alpha=0.5):
        """Soft projection toward positive cone."""
        g = self.metric_fn(phi)
        eigs = torch.linalg.eigvalsh(g)
        violation = torch.relu(-eigs).sum(dim=-1)
        
        needs_proj = violation > 0
        if not needs_proj.any():
            return phi
        
        t = torch.sigmoid(alpha * violation).unsqueeze(-1)
        t = t * needs_proj.float().unsqueeze(-1)
        phi_0 = self.phi_0.unsqueeze(0).expand_as(phi)
        
        return (1 - t) * phi + t * phi_0
    
    def get_phi_and_metric(self, x):
        phi = self.forward(x)
        g = self.metric_fn(phi)
        return phi, g

# Create model
model = G2VariationalNet(config).to(device)
n_params = sum(p.numel() for p in model.parameters())
print(f"Model parameters: {n_params:,}")

# Test forward
x_test = torch.randn(100, 7, device=device)
phi_test = model(x_test)
print(f"Input: {x_test.shape} -> Output: {phi_test.shape}")

## 5. Loss Functions

In [ ]:
def compute_torsion(phi, x, phi_fn):
    """Estimate ||d phi|| via finite differences."""
    batch = phi.shape[0]
    eps = 1e-4
    d_phi_sq = torch.zeros(batch, device=phi.device)
    
    for i in range(7):
        x_p, x_m = x.clone(), x.clone()
        x_p[:, i] += eps
        x_m[:, i] -= eps
        grad = (phi_fn(x_p) - phi_fn(x_m)) / (2 * eps)
        d_phi_sq += (grad ** 2).sum(dim=-1)
    
    return torch.sqrt(d_phi_sq / 7)


def variational_loss(model, x, weights, config):
    """Combined variational loss.
    
    L = w_torsion * (||T|| - kappa_T)^2 
      + w_det * (det(g) - 65/32)^2
      + w_pos * sum(relu(-eigenvalues))
    """
    phi, g = model.get_phi_and_metric(x)
    
    losses = {}
    
    # Torsion: target kappa_T = 1/61
    def phi_fn(x_new):
        return model(x_new)
    torsion = compute_torsion(phi, x, phi_fn)
    losses['torsion'] = ((torsion - config.kappa_T) ** 2).mean()
    losses['torsion_val'] = torsion.mean().item()
    
    # Determinant: target 65/32
    det_g = torch.det(g)
    losses['det'] = ((det_g - config.det_g_target) ** 2).mean()
    losses['det_val'] = det_g.mean().item()
    
    # Positivity: all eigenvalues > 0
    eigs = torch.linalg.eigvalsh(g)
    losses['pos'] = torch.relu(-eigs + 1e-6).sum(dim=-1).mean()
    losses['min_eig'] = eigs.min(dim=-1).values.mean().item()
    
    # Total
    total = (weights.get('torsion', 1.0) * losses['torsion'] +
             weights.get('det', 1.0) * losses['det'] +
             weights.get('pos', 1.0) * losses['pos'])
    
    losses['total'] = total
    
    return total, losses

print("Loss function defined.")

## 6. Training

In [ ]:
def sample_coords(n, device):
    return 2 * torch.rand(n, 7, device=device) - 1

def train(model, config, n_epochs=2000, device='cpu'):
    """Training with 4 phases."""
    
    phases = [
        {'name': 'init', 'epochs': n_epochs//4,
         'weights': {'torsion': 1.0, 'det': 0.5, 'pos': 2.0}},
        {'name': 'det', 'epochs': n_epochs//4,
         'weights': {'torsion': 1.0, 'det': 3.0, 'pos': 1.0}},
        {'name': 'torsion', 'epochs': n_epochs//4,
         'weights': {'torsion': 3.0, 'det': 1.0, 'pos': 1.0}},
        {'name': 'refine', 'epochs': n_epochs//4,
         'weights': {'torsion': 2.0, 'det': 2.0, 'pos': 0.5}},
    ]
    
    optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)
    scheduler = CosineAnnealingLR(optimizer, T_max=n_epochs)
    
    history = {'loss': [], 'det': [], 'torsion': [], 'phase': []}
    
    epoch = 0
    for phase in phases:
        print(f"\n--- Phase: {phase['name']} ({phase['epochs']} epochs) ---")
        
        for _ in range(phase['epochs']):
            model.train()
            x = sample_coords(config.batch_size, device)
            
            loss, losses = variational_loss(model, x, phase['weights'], config)
            
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
            
            history['loss'].append(losses['total'].item())
            history['det'].append(losses['det_val'])
            history['torsion'].append(losses['torsion_val'])
            history['phase'].append(phase['name'])
            
            if epoch % 200 == 0:
                det_err = 100 * abs(losses['det_val'] - config.det_g_target) / config.det_g_target
                tor_err = 100 * abs(losses['torsion_val'] - config.kappa_T) / config.kappa_T
                print(f"[{epoch:4d}] L={losses['total'].item():.4f} | "
                      f"det={losses['det_val']:.4f} ({det_err:.1f}%) | "
                      f"kappa={losses['torsion_val']:.4f} ({tor_err:.1f}%) | "
                      f"min_eig={losses['min_eig']:.4f}")
            
            epoch += 1
    
    return history

print("Training function defined.")

In [ ]:
# Train!
print("="*60)
print("GIFT v2.2 Variational G2 Training")
print("="*60)
print(f"Targets: det(g)={config.det_g_target}, kappa_T={config.kappa_T:.6f}")
print(f"Device: {device}")

model = G2VariationalNet(config).to(device)
history = train(model, config, n_epochs=2000, device=device)

## 7. Training Visualization

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

# Loss
ax = axes[0]
ax.semilogy(history['loss'])
ax.set_xlabel('Epoch')
ax.set_ylabel('Total Loss')
ax.set_title('Training Loss')
ax.grid(True, alpha=0.3)

# Determinant
ax = axes[1]
ax.plot(history['det'])
ax.axhline(config.det_g_target, color='r', linestyle='--', label=f'Target: {config.det_g_target}')
ax.set_xlabel('Epoch')
ax.set_ylabel('det(g)')
ax.set_title('Metric Determinant')
ax.legend()
ax.grid(True, alpha=0.3)

# Torsion
ax = axes[2]
ax.plot(history['torsion'])
ax.axhline(config.kappa_T, color='r', linestyle='--', label=f'Target: 1/61')
ax.set_xlabel('Epoch')
ax.set_ylabel('kappa_T')
ax.set_title('Torsion Magnitude')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 8. Validation

In [ ]:
model.eval()
n_val = 5000

with torch.no_grad():
    x = sample_coords(n_val, device)
    phi, g = model.get_phi_and_metric(x)
    det_g = torch.det(g)
    eigs = torch.linalg.eigvalsh(g)

det_np = det_g.cpu().numpy()
min_eig = eigs[:, 0].cpu().numpy()

print("="*60)
print("VALIDATION RESULTS")
print("="*60)
print(f"")
print(f"det(g):")
print(f"  Mean:   {det_np.mean():.6f}")
print(f"  Target: {config.det_g_target:.6f}")
print(f"  Error:  {100*abs(det_np.mean() - config.det_g_target)/config.det_g_target:.2f}%")
print(f"")
print(f"Positivity:")
print(f"  Min eigenvalue (mean): {min_eig.mean():.6f}")
print(f"  Fraction positive:     {100*(min_eig > 0).mean():.1f}%")
print(f"")
print(f"Eigenvalue spectrum:")
for i in range(7):
    eig_i = eigs[:, i].cpu().numpy()
    print(f"  lambda_{i}: {eig_i.mean():.4f} +/- {eig_i.std():.4f}")

In [ ]:
# Distributions
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

ax = axes[0]
ax.hist(det_np, bins=50, density=True, alpha=0.7)
ax.axvline(config.det_g_target, color='r', linestyle='--', linewidth=2,
           label=f'Target: {config.det_g_target:.4f}')
ax.axvline(det_np.mean(), color='g', linestyle='--', linewidth=2,
           label=f'Mean: {det_np.mean():.4f}')
ax.set_xlabel('det(g)')
ax.set_ylabel('Density')
ax.set_title('Metric Determinant Distribution')
ax.legend()

ax = axes[1]
ax.hist(min_eig, bins=50, density=True, alpha=0.7)
ax.axvline(0, color='r', linestyle='--', linewidth=2, label='Zero')
ax.set_xlabel('Minimum eigenvalue')
ax.set_ylabel('Density')
ax.set_title('Positivity Check')
ax.legend()

plt.tight_layout()
plt.show()

## 9. Mathematical Interpretation

> **Theorem (Conditional Existence)**
> 
> The PINN produces $\phi_{\text{num}}$ satisfying:
> - $|\det(g) - 65/32| < \delta_1$
> - $|\kappa_T - 1/61| < \delta_2$
> - $g(\phi) > 0$ (positive definite)
> 
> If the torsion functional $F[\phi] = ||d\phi||^2 + ||d^*\phi||^2$ is sufficiently small,
> by G2 deformation theory (Joyce), there exists an exact G2 structure nearby.

This is:
- **Numerical evidence** for existence of a specific G2 geometry
- **Not** a rigorous construction
- **Invitation** to mathematicians: prove this geometry exists

In [ ]:
# Final summary
print("="*60)
print("GIFT v2.2 VARIATIONAL G2 - SUMMARY")
print("="*60)
print(f"")
det_mean = det_np.mean()
det_err = 100 * abs(det_mean - config.det_g_target) / config.det_g_target
pos_frac = 100 * (min_eig > 0).mean()

print(f"Constraint Satisfaction:")
print(f"  det(g) = {det_mean:.6f} (target: {config.det_g_target}, error: {det_err:.2f}%)")
print(f"  Positivity: {pos_frac:.1f}% of samples")
print(f"")
print(f"GIFT v2.2 Predictions Used:")
print(f"  b2 = {config.b2_K7}, b3 = {config.b3_K7}")
print(f"  det(g) = 65/32 = {config.det_g_target}")
print(f"  kappa_T = 1/61 = {config.kappa_T:.6f}")
print(f"")
print(f"Status: {'PASSED' if det_err < 5 and pos_frac > 95 else 'NEEDS MORE TRAINING'}")